In [1]:
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import torch as tc
genders = ['female', 'male']
male = ['male']
female = ['female']

/home/wesley/.local/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
their_semantics = [
    'Subject',
    'Measuring station',
    'repetition no.',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Height (m)',
]
our_semantic = [
    'subject',
    'measures_station',
    'repetition',
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'stature', # 5.1.1
]

station_list = ["MOVE4D", "Expert measurer 1", "our"]

measures = pd.read_pickle(f'data/cleaned_measures.zip')
measures = measures[their_semantics]
measures.columns = our_semantic
our_measures = pd.read_pickle("data/our_measures.zip")
measures = pd.concat([our_measures, measures]).drop(columns=["gender", "stature"])
measures = measures[measures['measures_station'].isin(station_list)]

In [3]:
measures

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,repetition,measures_station,subject
0,890.102844,750.670166,1049.499756,628.432495,292.741119,325.192902,1,our,IEEEP2_01
1,895.18927,753.261353,1051.522095,625.345581,294.438354,326.743317,2,our,IEEEP2_01
2,903.245544,682.914734,947.78717,538.580383,274.551392,314.705322,1,our,IEEEP2_03
3,886.251953,664.944397,951.866211,537.98114,274.649323,316.283325,2,our,IEEEP2_03
4,1098.261108,981.130737,1141.172607,673.190918,343.884399,369.490021,1,our,IEEEP2_07
...,...,...,...,...,...,...,...,...,...
1992,1190.0,1120.0,1360.0,740.0,380.0,375.0,2,Expert measurer 1,IEEEP2_71
1997,931.73,864.896,988.994,572.149,319.426,370.596,1,MOVE4D,IEEEP2_72
1998,922.7,863.313,993.492,572.987,320.638,369.121,2,MOVE4D,IEEEP2_72
2007,930.0,825.0,985.0,565.0,305.0,355.0,1,Expert measurer 1,IEEEP2_72


# CALC ICC

In [4]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')['ICC'].loc[2]
        measures_of_station.append(icc)
    all_measures.append(measures_of_station)
our_icc = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [5]:
a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.995881,484.560754,71,72,4.732147e-77,"[0.99, 1.0]"
1,ICC2,Single random raters,0.995881,488.295694,71,71,3.445761e-76,"[0.99, 1.0]"
2,ICC3,Single fixed raters,0.995912,488.295694,71,71,3.445761e-76,"[0.99, 1.0]"
3,ICC1k,Average raters absolute,0.997936,484.560754,71,72,4.732147e-77,"[1.0, 1.0]"
4,ICC2k,Average random raters,0.997936,488.295694,71,71,3.445761e-76,"[1.0, 1.0]"
5,ICC3k,Average fixed raters,0.997952,488.295694,71,71,3.445761e-76,"[1.0, 1.0]"


In [6]:
our_icc.index = ['Expert', "MOVE4D", 'Our']
our_icc

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth
Expert,0.989452,0.989987,0.987519,0.980579,0.940527,0.979811
MOVE4D,0.994299,0.997657,0.999204,0.998120,0.995792,0.995974
Our,0.996868,0.998141,0.998908,0.998231,0.994649,0.995912


# CALC SEM

In [7]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')['ICC'].loc[2]
        sem = a[measure].std() * np.sqrt(1-icc)
        measures_of_station.append(sem)
    all_measures.append(measures_of_station)
our_sem = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [8]:
our_sem.index = ['Expert', "MOVE4D", 'Our']
our_sem.T.round(4)

,Expert,MOVE4D,Our
bust_chest_girth,11.7387,7.9139,5.6561
waist_girth,13.1368,6.4644,5.8106
hip_girth,11.6244,2.7550,3.0873
thigh_girth,8.9775,2.5773,2.6495
upper_arm_girth,10.0674,2.4972,2.5938
neck_girth,5.3662,2.3456,2.3739


# CALC PSD

In [9]:
def calculate_psd(measure):
    data = measures
    f1 = data['repetition'] == 1
    f2 = data['repetition'] == 2
    data = data[ f1 | f2 ][['subject', 'measures_station', 'repetition', measure]]
    data = data.pivot_table(values=measure, index=['subject', 'repetition'], columns="measures_station")
    data = data.dropna()
    

    column_wise = []
    for column1 in data.columns:
        
        row_wise = []
        for column2 in data.columns:
            if column1 == column2:
                row_wise.append(np.nan)
            else:
                d1 = data[column1]
                d2 = data[column2]
                var = (d1-d2).var()
                sem1 = our_sem[measure].loc[column1]**2
                sem2 = our_sem[measure].loc[column2]**2
                value = np.sqrt(var + (sem1/2) + (sem2/2))
                row_wise.append(value)
        column_wise.append(row_wise)
    data = pd.DataFrame(column_wise, index=data.columns, columns=data.columns)
    data.index.name = None
    return data

In [10]:
neck_result = calculate_psd(measure='neck_girth')
MX = pd.Series({"MOVE/Expert": neck_result.loc["MOVE4D"]["Expert measurer 1"]})
neck_result = neck_result.loc['our'][:-1].append(MX)

KeyError: 'Expert measurer 1'

In [ ]:
bust_result = calculate_psd(measure='bust_chest_girth')
MX = pd.Series({"MOVE/Expert": bust_result.loc["MOVE4D"]["Expert measurer 1"]})
bust_result = bust_result.loc['our'][:-1].append(MX)

In [ ]:
waist_result = calculate_psd(measure='waist_girth')
MX = pd.Series({"MOVE/Expert": waist_result.loc["MOVE4D"]["Expert measurer 1"]})
waist_result = waist_result.loc['our'][:-1].append(MX)

In [ ]:
hip_result = calculate_psd(measure='hip_girth')
MX = pd.Series({"MOVE/Expert": hip_result.loc["MOVE4D"]["Expert measurer 1"]})
hip_result = hip_result.loc['our'][:-1].append(MX)

In [ ]:
arm_result = calculate_psd(measure='upper_arm_girth')
MX = pd.Series({"MOVE/Expert": arm_result.loc["MOVE4D"]["Expert measurer 1"]})
arm_result = arm_result.loc['our'][:-1].append(MX)

In [ ]:
thigh_result = calculate_psd(measure='thigh_girth')
MX = pd.Series({"MOVE/Expert": thigh_result.loc["MOVE4D"]["Expert measurer 1"]})
thigh_result = thigh_result.loc['our'][:-1].append(MX)

In [ ]:
data = pd.DataFrame(
    [neck_result, bust_result, waist_result, hip_result, arm_result,thigh_result],
    index=our_semantic[3:-1]).round(2)
data.columns = ['Our vs Expert', "Our vs MOVE4D", 'MOVE vs Expert']
data.to_csv("output/our_psd.csv")
data

In [ ]:
def calculate_mad():
    all_measures = []
    for station in station_list:
        measures_of_station = []
        a = measures[measures['measures_station'] == station]
        f1 = a['repetition'] == 1
        f2 = a['repetition'] == 2
        for measure in our_semantic[3:-1]:
            mean = np.nanmean(abs(a[f1][measure].to_numpy() - a[f2][measure].to_numpy()))
            measures_of_station.append(mean)
        all_measures.append(measures_of_station)
    return pd.DataFrame(all_measures, columns=our_semantic[3:-1], index=['MOVE4D','Expert','Our']).T

In [ ]:
calculate_mad().round(4).to_csv("output/our_mad.csv")
calculate_mad().round(4)

In [ ]:
grand_mean = measures.mean()
grand_mean[:-1]

In [ ]:
def calculate_msd():

    all_measures = []

    for station in station_list:
        measures_of_station = []
        a = measures[measures['measures_station'] == station]
        for measure in our_semantic[3:-1]:
            measured = a[measure].to_numpy()
            ground_thruth = grand_mean[measure]
            msd = np.nanmean(measured - ground_thruth)
            measures_of_station.append(msd)
        all_measures.append(measures_of_station)
    return pd.DataFrame(all_measures, columns=our_semantic[3:-1], index=station_list).T
calculate_msd().round(4).to_csv("output/our_msd.csv")
calculate_msd().round(4)